In [1]:
import ufal.udpipe
from ufal.udpipe import Trainer
import os
import conllu

In [2]:
#Implementing a processing pipeline, which inclides tokenization, morphological tagging and syntactic parsing
#Source: https://github.com/ufal/udpipe/tree/master/bindings/python/examples

class Model:
    def __init__(self, model):
        """Load given model."""
        self.model = model
        if not self.model:
            raise Exception("Cannot load UDPipe model from file '%s'" % model)

    def tokenize(self, text):
        """Tokenize the text and return list of ufal.udpipe.Sentence-s."""
        tokenizer = self.model.newTokenizer(self.model.DEFAULT)
        if not tokenizer:
            raise Exception("The model does not have a tokenizer")
        return self._read(text, tokenizer)

    def read(self, text, format):
        """Load text in the given format (conllu|horizontal|vertical) and return list of ufal.udpipe.Sentence-s."""
        input_format = ufal.udpipe.InputFormat.newInputFormat(format)
        if not input_format:
            raise Exception("Cannot create input format '%s'" % format)
        return self._read(text, input_format)

    def _read(self, text, input_format):
        input_format.setText(text)
        error = ufal.udpipe.ProcessingError()
        sentences = []

        sentence = ufal.udpipe.Sentence()
        while input_format.nextSentence(sentence, error):
            sentences.append(sentence)
            sentence = ufal.udpipe.Sentence()
        if error.occurred():
            raise Exception(error.message)

        return sentences

    def tag(self, sentence):
        """Tag the given ufal.udpipe.Sentence (inplace)."""
        self.model.tag(sentence, self.model.DEFAULT)

    def parse(self, sentence):
        """Parse the given ufal.udpipe.Sentence (inplace)."""
        self.model.parse(sentence, self.model.DEFAULT)

    def write(self, sentences, format):
        """Write given ufal.udpipe.Sentence-s in the required format (conllu|horizontal|vertical)."""

        output_format = ufal.udpipe.OutputFormat.newOutputFormat(format)
        output = ''
        for sentence in sentences:
            output += output_format.writeSentence(sentence)
        output += output_format.finishDocument()
        return output
    
def process(raw_model, text):
    model = Model(raw_model)
    sentences = model.tokenize(text)
    for sent in sentences:
        model.tag(sent)
        model.parse(sent)
    output = model.write(sentences, "conllu")
    return output

In [3]:
#Parsing the test text using the model pre-trained on UD2017 data 
model_conll17 = ufal.udpipe.Model.load('russian-syntagrus-ud-2.3-181115.udpipe')
with open('UD_Russian-SynTagRus/ru_syntagrus-ud-dev.txt', 'r', encoding = 'utf-8') as f:
    input_text = f.read()[:10000]
    parsed = process(model_conll17, input_text)

In [4]:
print(parsed[:1000])

# newdoc
# newpar
# sent_id = 1
# text = Алгоритм, от имени учёного аль-Хорезми, - точный набор инструкций, описывающих порядок действий исполнителя для достижения результата решения задачи за конечное время.
1	Алгоритм	алгоритм	NOUN	_	Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing	0	root	_	SpaceAfter=No
2	,	,	PUNCT	_	_	4	punct	_	_
3	от	от	ADP	_	_	4	case	_	_
4	имени	имя	NOUN	_	Animacy=Inan|Case=Gen|Gender=Neut|Number=Sing	1	conj	_	_
5	учёного	учёной	NOUN	_	Animacy=Anim|Case=Gen|Gender=Masc|Number=Sing	4	nmod	_	_
6	аль	аль	PART	_	_	8	compound	_	SpaceAfter=No
7	-	-	PUNCT	_	_	6	punct	_	SpaceAfter=No
8	Хорезми	Хорезй	ADJ	_	Case=Ins|Degree=Pos|Number=Plur	4	obl	_	SpaceAfter=No
9	,	,	PUNCT	_	_	8	punct	_	_
10	-	-	PUNCT	_	_	12	punct	_	_
11	точный	точный	ADJ	_	Case=Nom|Degree=Pos|Gender=Masc|Number=Sing	12	amod	_	_
12	набор	набор	NOUN	_	Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing	4	nsubj	_	_
13	инструкций	инструкция	NOUN	_	Animacy=Inan|Case=Gen|Gender=Fem|Number=Plur	12	nmod	_	SpaceAfter=No
14

In [5]:
#loading 2018 UD data for subsequent model training
import codecs
with codecs.open('UD_Russian-SynTagRus/ru_syntagrus-ud-train.conllu', 'r', encoding='utf-8', errors='replace') as f:
    text = f.read()
    #в следующих трех строчках я попытался заменить все не-ascii символы, но ошибка все равно возникает
    import string
    printable = set(string.printable)
    filter(lambda x: x in printable, text)
    input_format = ufal.udpipe.InputFormat.newInputFormat('conllu')
    input_format.setText(text)
    error = ufal.udpipe.ProcessingError()
    data18 = []
    sentence = ufal.udpipe.Sentence()
    while input_format.nextSentence(sentence, error):
        data18.append(sentence)
        sentence = ufal.udpipe.Sentence()
    if error.occurred():
        raise Exception(error.message)

In [6]:
#Training model based on 2018 UD data
model_conll18 = Trainer.train('morphodita_parsito', data18[:1000], data18[1001:1500], 'epochs=3', Trainer.DEFAULT, 'iterations=3')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8b in position 25: invalid start byte

In [7]:
with open('UD_Russian-SynTagRus/ru_syntagrus-ud-dev.txt', 'r', encoding = 'utf-8') as f:
    input_text = f.read()[:10000]
    parsed = process(model_conll18, input_text)

NameError: name 'model_conll18' is not defined

In [12]:
#Loading test data - the golden standard for further models to be compared to
with open('UD_Russian-SynTagRus/ru_syntagrus-ud-dev.conllu', 'r', encoding = 'utf-8') as f:
    

[TokenList<Алгоритм, ,, от, имени, учёного, аль, -, Хорезми, ,, -, точный, набор, инструкций, ,, описывающих, порядок, действий, исполнителя, для, достижения, результата, решения, задачи, за, конечное, время, .>, TokenList<В, старой, трактовке, вместо, слова, ", порядок, ", использовалось, слово, ", последовательность, ", ,, но, по, мере, развития, параллельности, в, работе, компьютеров, слово, ", последовательность, ", стали, заменять, более, общим, словом, ", порядок, ", .>, TokenList<Это, связано, с, тем, ,, что, работа, каких-то, инструкций, алгоритма, может, быть, зависима, от, других, инструкций, или, результатов, их, работы, .>, TokenList<Таким, образом, ,, некоторые, инструкции, должны, выполняться, строго, после, завершения, работы, инструкций, ,, от, которых, они, зависят, .>, TokenList<Независимые, инструкции, или, инструкции, ,, ставшие, независимыми, из-за, завершения, работы, инструкций, ,, от, которых, они, зависят, ,, могут, выполняться, в, произвольном, порядке, ,, пар